# Run PT-JPL model adapted for arid lands 

This notebook demonstrates the use of `geeet.ptjpl` - a module to compute evaporation components (ET, or crop water use) using the PT-JPL model combined with satellite and climate reanalysis data from Google Earth Engine (GEE). 

To use this notebook, you will need to install [geemap](https://geemap.org/installation/).

In [1]:
import geemap
geemap.ee_initialize()
import ee

## GEE data

In this demonstration, we use data from the [ECMWF/ERA5_LAND/HOURLY](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY#bands), [NOAA/GFS0P25](https://developers.google.com/earth-engine/datasets/catalog/NOAA_GFS0P25#bands), and [MODIS/MCD43A4_006_NDVI](https://developers.google.com/earth-engine/datasets/catalog/MODIS_MCD43A4_006_NDVI) datasets. Below we define the input images and visualize them on a map:

In [6]:
date_time_str='2017-04-15T13:00:00' 
date_time = ee.Date(date_time_str)

lonlat=[38.5, 30] 
date = ee.Date(date_time)
geom = ee.Geometry.Point(lonlat[0], lonlat[1]) # lon/lat
region = geom.buffer(10000) # 10-km radius around the point 

RH = ee.Image(
  ee.ImageCollection('NOAA/GFS0P25')
    .filterBounds(region)
    .filterDate(date,date.advance(1,'month'))
    .sort('system:time_start')
    .first()
  ).select('relative_humidity_2m_above_ground').rename('relative_humidity')

era5_img = ee.Image(
  ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
    .filterBounds(region)
    .filterDate(date,date.advance(1,'hour'))
    .sort('system:time_start')
    .first()
  )
NDVI = ee.ImageCollection('MODIS/MCD43A4_006_NDVI') \
                  .filter(ee.Filter.date('2018-04-01', '2018-05-01')).first() \
                  .select("NDVI")

NDVI_one_year = ee.ImageCollection('MODIS/MCD43A4_006_NDVI') \
                  .filter(ee.Filter.date('2016-01-01', '2016-12-01')) \
                  .select("NDVI")
# NDVI_one_year will be used to compute f_aparmax:
from geeet.ptjpl import add_fapar
fapar_max = NDVI_one_year.map(add_fapar).select('fapar').reduce(ee.Reducer.max())

t2m = 'temperature_2m' # in K
sfp = 'surface_pressure' # in Pa (see https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY#bands)
net = 'surface_net_solar_radiation_hourly' # in J/m2 (see https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY#bands)

tair = era5_img.select(t2m).subtract(273.15).rename('temperature_C') # in celsius
surf = era5_img.select(sfp).divide(1000.0).rename('surface_pressure_KPa')  # convert from Pa to Kpa 
rnet = era5_img.select(net).divide(3600.0).rename('net_radiation')  # convert from J/m2 to W/m2 (this variable is accumulated from the start of the simulation - 00:00 every day)


import geemap.colormaps as cm
ndvi_pal = cm.palettes.ndvi
pal = cm.palettes.YlOrRd
pal_viridis = cm.palettes.viridis


et_inputs = RH.addBands(tair).addBands(surf).addBands(rnet).addBands(NDVI).addBands(fapar_max)
doy = date_time.getRelative('day', 'year').add(1)
time = date_time.getRelative('hour', 'day').add(3)  # offset of 3 hours to obtain local time in this region.
et_inputs = et_inputs.set('doy',  doy)
et_inputs = et_inputs.set('time', time)

# Finally, apply a mask so that we collect data only from the specified region:
img_mask = ee.Image.constant(1).clip(region).mask()
def mask_img(img):
    return img.mask(img_mask)
et_inputs = mask_img(et_inputs)
# (if you want to apply the model to the entire tile, comment the previous line


Map = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=12)
Map.addLayer(et_inputs.select('relative_humidity'), {'min':0, 'max':100, 'palette':["440154","3a528b","20908d","5dc962","fde725"], 'opacity': 0.8}, 'Relative humidity')
Map.addLayer(et_inputs.select('temperature_C'), {'min':0, 'max':35, 'palette':pal, 'opacity': 0.8}, 'Air temperature (2m)')
Map.addLayer(et_inputs.select('surface_pressure_KPa'), {'min':90, 'max':110, 'palette':pal, 'opacity': 0.8}, 'Surface pressure (KPa)')
Map.addLayer(et_inputs.select('net_radiation'), {'min':0, 'max':500, 'palette':pal, 'opacity': 0.8}, 'Net solar radiation (W/m2)')
Map.addLayer(et_inputs.select('NDVI'), {'min':0, 'max':1, 'palette':ndvi_pal, 'opacity': 0.8}, 'NDVI')
Map.addLayer(et_inputs.select('fapar_max'), {'min':0, 'max':1, 'palette':pal, 'opacity': 0.8}, 'F_aparmax')

Map

Map(center=[30, 38.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Run the PT-JPL model

We now run the model and add the results to the map:

In [8]:
from geeet.ptjpl import ptjpl_arid
ET = ptjpl_arid(img = et_inputs)
Map.addLayer(ET.select(['H']), {'min':0, 'max':500, 'palette':pal_viridis, 'opacity': 0.8}, 'Sensible heat flux')
Map.addLayer(ET.select(['G']), {'min':0, 'max':500, 'palette':pal_viridis, 'opacity': 0.8}, 'Ground heat flux')
Map.addLayer(ET.select(['Rn']), {'min':0, 'max':500, 'palette':pal_viridis, 'opacity': 0.8}, 'Net radiation')
Map.addLayer(ET.select(['LEs']), {'min':0, 'max':500, 'palette':pal_viridis, 'opacity': 0.8}, 'LE soil')
Map.addLayer(ET.select(['LEc']), {'min':0, 'max':500, 'palette':pal_viridis, 'opacity': 0.8}, 'LE canopy')
Map.addLayer(ET.select(['LEi']), {'min':0, 'max':500, 'palette':pal_viridis, 'opacity': 0.8}, 'LE interception')
Map.addLayer(ET.select(['LE']), {'min':0, 'max':500, 'palette':pal_viridis, 'opacity': 0.8}, 'LE')

Use the inspector (i) tool in the map above to display the values for one pixel. Then copy the values for the inputs below to run the model with numerical values - this is useful to double check the outputs in the image. It also demonstrates the hybrid nature of the function - if given numbers, it returns numbers, if given ee.Images, it returns ee.Images. 

In [ ]:
# eot_params: Day of year, time, longitude, standard meridian
ptjpl_arid(RH=25.3, Temp_C=24.3442, Press=94.524, Rn=374.8117, NDVI=0.7588, F_aparmax=0.7295, doy = 105, time = 11, longitude = 38.4381)    
# The returned values are LE, LEc, LEs, LEi, H, G, Rn

Example output of the inspector tool:
```
Point (38.4381, 30.1938) at 38m/px
Relative humidity: Image (1 band)
  relative_humidity_2m_above_ground: 25.30000114440918
Air temperature (2m): Image (1 band)
  temperature_2m: 24.344262695312523
Surface pressure (KPa): Image (1 band)
  surface_pressure: 94.52407421875
Net solar radiation (W/m2): Image (1 band)
  surface_net_solar_radiation: 374.81166666666667
NDVI: Image (1 band)
  NDVI: 0.7588107585906982
F_aparmax: Image (1 band)
  fapar_max: 0.7295318901538849
Sensible heat flux: Image (1 band)
  H: 98.33337358552566
Ground heat flux: Image (1 band)
  G: 23.486755569509857
Net radiation: Image (1 band)
  Rn: 374.81166666666667
LE soil: Image (1 band)
  LEs: 2.764964916235911
LE canopy: Image (1 band)
  LEc: 249.11492649022074
LE interception: Image (1 band)
  LEi: 1.1116481816579635
LE: Image (1 band)
  LE: 252.9915395881146
```

## References

    Allen, R.G., Pereira, L.S., Raes, D., Smith, M. 
        Crop evapotranspiration —guidelines for computing crop water requirements
        (1998) FAO Irrigation and drainage paper 56. Food and Agriculture 
        Organization, Rome, pp. 35-39. 
        http://www.fao.org/docrep/x0490e/x0490e00.htm

    Aragon, B., Houborg, R., Tu, K., Fisher, J.B., McCabe, M.
        Cubesats enable high spatiotemporal retrievals of crop-water use for 
        precision agriculture (2018)
        Remote Sensing, 10 (12), art. no. 1867.
        http://dx.doi.org/10.3390/rs10121867   

    Campbell, G. S., & Norman, J. M.
        Introduction to environmental biophysics (2nd ed.) (1998)
        New York: Springer, pp. 168-169
        http://dx.doi.org/10.1007/978-1-4612-1626-1

    Carlson, T.N., Capehart, W.J., Gillies, R.R.
        A new look at the simplified method for remote sensing of daily 
        evapotranspiration (1995) 
        Remote Sensing of Environment, 54 (2), pp. 161-167.
        http://dx.doi.org/10.1016/0034-4257(95)00139-R
        
    Choudhury, B.J., Ahmed, N.U., Idso, S.B., Reginato, R.J., Daughtry, C.S.T.
        Relations between evaporation coefficients and vegetation indices 
        studied by model simulations (1994) 
        Remote Sensing of Environment, 50 (1), pp. 1-17.
        http://dx.doi.org/10.1016/0034-4257(94)90090-6
        
    Fisher, J.B., Tu, K.P., Baldocchi, D.D.
        Global estimates of the land-atmosphere water flux based on monthly
        AVHRR and ISLSCP-II data, validated at 16 FLUXNET sites (2008)
        Remote Sensing of Environment, 112 (3), pp. 901-919.
        http://dx.doi.org/10.1016/j.rse.2007.06.025

    Potter, C.S., Randerson, J.T., Field, C.B., Matson, P.A., Vitousek, P.M.,
    Mooney, H.A., Klooster, S.A.
        Terrestrial ecosystem production: A process model based on global 
        satellite and surface data (1993) 
        Global Biogeochemical Cycles, 7 (4), pp. 811-841. 
        http://dx.doi.org/10.1029/93GB02725
        
    Priestley, C.H.B. and Taylor, R.J.
        On the Assessment of Surface Heat Flux and Evaporation Using Large Scale 
        Parameters (1972) Monthly Weather Review, 100, 81-92.
        http://dx.doi.org/10.1175/1520-0493(1972)100<0081:OTAOSH>2.3.CO;2
        
    Joseph A. Santanello Jr. and Mark A. Friedl.
        Diurnal Covariation in Soil Heat Flux and Net Radiation (2003)
        J. Appl. Meteor., 42, pp. 851-862.
        Remote Sensing of Environment, 112 (3), pp. 901-919.
        http://dx.doi.org/10.1175/1520-0450(2003)042<0851:DCISHF>2.0.CO;2